In [1]:
import math
import numpy as np
import codecs, json 
import cv2
import matplotlib.pyplot as plt
from sklearn import mixture
from scipy import linalg as la
from scipy import stats as st
from scipy.interpolate import griddata
from scipy.interpolate import interp1d

In [2]:
c_error = np.array([[ 0.53,  0.17,  0.27,  0.27,  0.33,  0.33,  0.31,  0.28],
       [ 0.19, -0.05,  0.02,  0.05,  0.11,  0.06,  0.03, -0.01],
       [ 0.21, -0.  ,  0.13,  0.08,  0.14,  0.14,  0.04, -0.05],
       [ 0.27, -0.05,  0.09,  0.04,  0.15,  0.19,  0.05, -0.05],
       [ 0.22, -0.24, -0.14, -0.  ,  0.08,  0.16,  0.03, -0.05],
       [ 0.01, -0.4 , -0.28, -0.1 , -0.05,  0.07,  0.03, -0.05],
       [-0.23, -0.42, -0.3 , -0.22, -0.13,  0.01,  0.03, -0.08],
       [-0.21, -0.46, -0.28, -0.24, -0.17,  0.01, -0.08, -0.13]])

c_maxnoise = np.array([[0.42, 0.18, 0.24, 0.26, 0.36, 0.42, 0.45, 0.44],
       [0.22, 0.21, 0.26, 0.31, 0.2 , 0.28, 0.22, 0.36],
       [0.18, 0.14, 0.12, 0.37, 0.12, 0.14, 0.29, 0.32],
       [0.2 , 0.3 , 0.29, 0.13, 0.23, 0.15, 0.3 , 0.23],
       [0.31, 0.1 , 0.14, 0.28, 0.29, 0.31, 0.18, 0.38],
       [0.42, 0.23, 0.37, 0.31, 0.18, 0.34, 0.24, 0.3 ],
       [0.29, 0.26, 0.34, 0.3 , 0.15, 0.3 , 0.35, 0.28],
       [0.39, 0.46, 0.32, 0.42, 0.43, 0.37, 0.46, 0.4 ]])

def read_tempdata(rn, path):
    arr = np.zeros([rn,8,8])
    with open(path) as f:
        for k in range(rn):
            for i in range(8):
                line = f.readline()
                t = [float(x) for x in line.split()]
                arr[k,i] = t
            line = f.readline()
#     arr = arr - c_error
    arr = np.amax(arr, axis = 0) - c_maxnoise
    return arr

def read_gtempdata(rn,gpath,num):
    garr = np.zeros([num,8,8])
    for n in range(num):
        path = gpath + str(n+1) +'.txt'
        garr[n] = read_tempdata(rn, path)
    return garr

In [3]:
points = [(math.floor(ix / 8), (ix % 8)) for ix in range(0, 64)]
grid_x, grid_y = np.mgrid[0:7:64j, 0:7:64j]

def show_temp(arr, mintemp = 26, maxtemp = 32):
    plt.figure(figsize=(6,5))
    bicubic = griddata(points, arr.reshape(-1), (grid_x, grid_y), method='cubic')
    pcm = plt.pcolormesh(np.flip(bicubic,0), cmap='jet', vmin = mintemp, vmax = maxtemp)
    plt.colorbar(pcm, fraction=0.175,pad=0.02)
    plt.axis('off')
    plt.show()
    
    return np.amax(bicubic)


def show_gtemp(garr,ln,weight = None, mintemp = 26, maxtemp = 32):
    fig, axs = plt.subplots(ln, ln, figsize=(15, 15))
    amax = np.zeros((ln,ln))
    for n in range(ln**2):
        col = int(n/ln)
        row = int(n%ln)
        bicubic = griddata(points, garr[n].reshape(-1), (grid_x, grid_y), method='cubic')
        if np.any(weight) != None:
            wbicubic = griddata(points, weight[n].reshape(-1)*80, (grid_x, grid_y), method='cubic')/80
            wbicubic[wbicubic < 0] = 0
            wbicubic[wbicubic > 1] = 1
            bicubic = bicubic*(1-wbicubic) + np.amin(garr)*wbicubic
        ax = axs[col,row]
        pcm = ax.pcolormesh(np.flip(bicubic,0), cmap='jet', vmin = mintemp, vmax = maxtemp)
        ax.set_axis_off()
        amax[col,row] = np.amax(bicubic)
    
    fig.colorbar(pcm, ax= axs[:], fraction=0.0175,pad=0.02)

    plt.show()
    print(amax.round(2))

In [4]:
def get_weight(garr,arrbg):
    diff = garr.reshape(-1,64)-arrbg.reshape(-1)
    
    weight = np.ones(diff.shape)
    for i in range(diff.shape[0]):
        n = 0
        n_skew = 10000
        bicubic = np.around(griddata(points, diff[i], (grid_x, grid_y), method='cubic').reshape(-1) * 4)/4.
        sort_diff = np.unique(bicubic)
        mode =  0
        start = 0
        for j in range(int(sort_diff.shape[0]/2)):
            num = bicubic[bicubic == sort_diff[j]].shape[0]
            if (num > mode):
                mode = num
                start = j
        for j in range(max(2,start),int(sort_diff.shape[0]/2)):
            t_skew = abs(st.skew(bicubic[bicubic < sort_diff[j]] ))
            if t_skew < n_skew:
                n_skew = t_skew
                n = sort_diff[j]
        weight[i,diff[i] >= n] = 0 
    
    return weight.reshape(-1,8,8)

In [34]:
temp_path = 'warm'
datapath = 'train/' + temp_path + '/output-4/'
garr = read_gtempdata(100,datapath,3**2)
ans = 0
for i in range(3):
    for j in range(3):
        temp = np.mean(garr[:,1+i:4+i,1+j:4+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/9)
print("")

ans = 0
for i in range(3):
    for j in range(3):
        temp = np.mean(garr[:,2+i:3+i,2+j:3+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/9)

28.58
29.1
29.24
28.83
29.47
29.56
28.78
29.51
29.64

29.189999999999998

28.85
29.41
29.69
28.74
29.95
29.74
28.89
29.86
30.13

29.473333333333333


In [35]:
datapath = 'train/' + temp_path + '/output-3/'
garr = read_gtempdata(100,datapath,4**2)
ans = 0
for i in range(4):
    for j in range(4):
        temp = np.mean(garr[:,1+i:3+i,1+j:3+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/16)
print("")

ans = 0
for i in range(4):
    for j in range(4):
        temp = np.mean(garr[:,2+i,2+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/16)

27.51
27.69
27.81
27.73
27.63
27.94
28.03
27.84
27.74
28.1
28.21
27.94
27.81
28.28
28.59
28.37

27.951249999999998

27.79
27.82
28.0
27.63
27.85
28.31
27.99
27.76
28.06
28.19
28.35
27.66
28.02
28.85
28.96
28.5

28.108750000000004


In [31]:
datapath = 'train/' + temp_path + '/output-2/'
garr = read_gtempdata(100,datapath,5**2)
ans = 0
for i in range(5):
    for j in range(5):
        temp = np.mean(garr[:,1+i:2+i,1+j:2+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/25)

27.19
27.12
27.3
27.22
27.27
27.37
27.49
27.29
27.33
27.24
27.26
27.53
27.63
27.2
27.33
27.67
27.74
27.66
27.72
27.31
27.17
27.5
27.92
28.14
27.98

27.463199999999997


In [32]:
datapath = 'train/' + temp_path + '/output-1/'
garr = read_gtempdata(100,datapath,6**2)
ans = 0
for i in range(6):
    for j in range(6):
        temp = np.mean(garr[:,1+i,1+j]).round(2)
        ans += temp
        print(temp)
print("")
print(ans/36)

27.13
27.01
27.06
27.02
27.0
27.09
27.42
27.43
27.05
27.14
27.12
27.01
27.33
27.49
27.4
26.98
27.13
26.95
27.71
27.67
27.26
27.31
26.93
27.03
27.27
27.39
27.67
27.81
27.72
27.29
27.31
27.53
27.69
28.02
27.92
27.55

27.328888888888876
